In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np

In [5]:
class contextual_bandit():
    def __init__(self):
        self.state = 0
        
        self.bandits = np.array([[0.2,0,-0.0,-5],[0.1,-5,1,0.25],[-5,5,5,5]])
        self.num_bandits = self.bandits.shape[0]
        self.num_actions = self.bandits.shape[1]
        #the three bandits with four arms each. we want to settle on actions 4,2,1 respectively)
    
    def getBandit(self):
        self.state = np.random.randint(0,self.num_bandits)
        return self.state
        #return random bandit

    def pullArm(self, action):
        bandit = self.bandits[self.state,action]
        result = np.random.randn(1)
        
        if result > bandit:
            return 1
        else:
            return -1
        #sample based on which lever on which bandit (action on state)


In [6]:
class agent():
    def __init__(self, lr, s_size,a_size):
        #not sure what the fuck this is
        #jk it's the constructor
        
        self.state_in = tf.placeholder(shape=[1], dtype=tf.int32)
            #placeholder for current state input
        state_in_OH = slim.one_hot_encoding(self.state_in,s_size)
            #convert to one-hot (1x3 vector with nth bit as 1 where n = bandit #)
        #print (state_in_OH)
        output = slim.fully_connected(state_in_OH,a_size,\
            biases_initializer=None,activation_fn=tf.nn.sigmoid,weights_initializer=tf.ones_initializer())
            #fully connected nn -> 3 inputs, 4 outputs, so we learn 12 weights
            
        self.output = tf.reshape(output, [-1])
            #? reshape output from vertical to horizontal? not sure
            # A. reshape to a 4-weight 1D vector
        
        self.chosen_action = tf.argmax(self.output, 0)
            # take argmax as chosen action
        
        
        self.reward_holder = tf.placeholder(shape=[1], dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[1], dtype=tf.int32)
            #placeholders for the reward and action that resulted in it
        
        self.responsible_weight = tf.slice(self.output, self.action_holder, [1])
        
        self.loss = -(tf.log(self.responsible_weight)*self.reward_holder)
            #minimize the negative log weight * reward -> maximize weight of highest reward
            #q. how does the optimizer select the right weights to change?
                #a. backprop. but i don't have an intuitive understanding of how they change
            
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
        self.update = optimizer.minimize(self.loss)

In [7]:
tf.reset_default_graph()

cBandit = contextual_bandit()
    #load bandits

myAgent = agent(lr=0.001,s_size=cBandit.num_bandits, a_size=cBandit.num_actions)
weights = tf.trainable_variables()[0]
    #load all trainable variables into weights.
    #...why do we do it like this?

total_episodes = 10000
    #damn that's a lot

total_reward = np.zeros([cBandit.num_bandits,cBandit.num_actions])
    #3x4 scoreboard

e = 0.1

init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        s = cBandit.getBandit()
            #observe our random state
        
        if np.random.rand(1) < 3:
            action = np.random.randint(cBandit.num_actions)
        else:
            action = sess.run(myAgent.chosen_action, feed_dict = {myAgent.state_in:[s]})
            #choose random action, or feed in observed state to get chosen action (corr. to largest weight)
            # run sess
        
        reward = cBandit.pullArm(action)
            #get reward
        feed_dict={myAgent.reward_holder:[reward],myAgent.action_holder:[action],myAgent.state_in:[s]}
        _,ww = sess.run([myAgent.update, weights], feed_dict=feed_dict)
            #train net...
            
        total_reward[s, action] += reward
        
        if i % 500 == 0:
            print ("Mean reward for each of the " + str(cBandit.num_bandits) + " bandits: " + str(np.mean(total_reward,axis=1)))
        i+=1
for a in range(cBandit.num_bandits):
    print ("The agent thinks action " + str(np.argmax(ww[a])+1) + " for bandit " + str(a+1) + " is the most promising....")
    if np.argmax(ww[a]) == np.argmin(cBandit.bandits[a]):
        print ("...and it was right!")
    else:
        print ("...and it was wrong!")
        
        

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Mean reward for each of the 3 bandits: [ 0.25  0.    0.  ]
Mean reward for each of the 3 bandits: [ 11.5    6.   -19.25]
Mean reward for each of the 3 bandits: [ 20.25   7.5  -44.  ]
Mean reward for each of the 3 bandits: [ 29.75   7.75 -68.25]
Mean reward for each of the 3 bandits: [ 40.5    8.75 -93.5 ]
Mean reward for each of the 3 bandits: [  54.5     3.   -113.75]
Mean reward for each of the 3 bandits: [  60.25    5.25 -130.75]
Mean reward for each of the 3 bandits: [  67.75    2.   -143.  ]
Mean reward for each of the 3 bandits: [  83.75    7.   -163.5 ]
Mean reward for each of the 3 bandits: [  92.25   11.5  -181.5 ]
Mean reward for each of the 3 bandits: [ 105.25   16.5  -200.  ]
Mean reward for each of the 3 bandits: [ 121.75   20.   -223.5 ]
Mean reward for each of the 3 bandits: [ 130.     20.5  -245.25]
Mean reward for each of the 3 bandits: [ 138.5    22.25 -261.5 ]
Mean reward for each of the 3 band